## Introduction

Tao đang muốn test run full workflow deep learning trên spark thông qua py-spark cell. deepleaning với thư viện tensorflow, keras. nội dung thực hiện bao gồm load một csv file bank gồm:
[id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit]
load và review, sau đó drop some field sau đó làm thêm một số cái linh sờ tinh như extract feature transform bla bla chung quy là test spark pineline. Hồi sau sẽ thêm nhiều cái như thay vì load từ csv sẽ load từ chanel trong kafka streaming và read file từ hdfs. bla bla =))


<div class="alert alert-block alert-info">
<b>Step 1:</b> Import Libraries
</div>

In [ ]:
# Spark Session, Pipeline, Functions, and Metrics
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline
import pandas as pd
import os

# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.2,' \
#                                     + 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.2,' \
#                                     + 'org.apache.kafka:kafka-clients:3.1.0,org.apache.spark:spark-core_2.12:3.0.2,' \
#                                     + 'org.apache.spark:spark-streaming_2.12:3.0.2,' \
#                                     + 'org.apache.spark:spark-token-provider-kafka-0-10_2.12:3.0.2,' \
#                                     + 'org.apache.hadoop:hadoop-core:1.2.1' \
#                                     + ' pyspark-shell'

<div class="alert alert-block alert-info">
<b>Step 2:</b> Start Spark Session
</div>

In [ ]:
TOPIC = 'kafka-sample'
SPARK_SERVER="spark://spark-master:7077"

ss  = SparkSession.builder.master('local[2]') \
                  .appName("test") \
                  .config("spark.executor.memory", "2g") \
                  .config("spark.driver.memory", "2g") \
                  .config("spark.executor.cores", "2") \
                  .config("spark.driver.cores", "2") \
                  .getOrCreate()
ss
# sc = ss.sparkContext
# sql_c = SQLContext(sc)
# sc

<div class="alert alert-block alert-info">
<b>Step 3:</b> Load and Preview Data
</div>

Tại đây, tao sẽ tải dữ liệu. Dữ liệu tao sử dụng đến từ một cuộc thi Kaggle [Kaggle competition](https://www.kaggle.com/janiobachmann/bank-marketing-dataset). Đó là một tập dữ liệu ngân hàng điển hình. Ở đây tao sử dụng tham số invSchema giúp xác định các loại tính năng khi tải dữ liệu. Theo tài liệu PySpark [PySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html), điều này "yêu cầu thêm một lần chuyển dữ liệu". Vì dữ liệu ngân hàng mà tao đang tải chỉ có ~ 11 nghìn lần quan sát nên không mất nhiều thời gian, nhưng có thể đáng chú ý nếu mày có một tập dữ liệu rất lớn.

Sau khi tải dữ liệu, chúng tao có thể thấy lược đồ và các loại tính năng khác nhau. Tất cả các tính năng của chúng tao là kiểu chuỗi hoặc số nguyên. Sau đó, chúng tao xem trước 5 quan sát đầu tiên. tao khá quen thuộc với thư viện Pandas python nên qua ví dụ này, mày sẽ thấy tao sử dụng toPandas () để chuyển đổi khung dữ liệu spark thành khung dữ liệu gấu trúc và thực hiện một số thao tác. Không đúng không sai, chỉ là dễ dàng hơn cho tao.

Cuối cùng, chúng tao sẽ bỏ 2 cột ngày vì chúng tao sẽ không sử dụng các cột đó trong mô hình học sâu của mình. Chúng có thể rất quan trọng và được kỳ công bởi tao quyết định thả tất cả chúng lại với nhau. ồ tao đc kết quả như mong muốn lè =)))

<div class="alert alert-block alert-warning">
tại đây tao gặp một vấn đề chưa giải quyết đc là chưa dùng spark để write file tới hdfs đc. méo hiểu kiểu gì lun ...(-.-)! maybe là tao sẽ kiểm tra nó sau. nên hiện tại tao dùng python viết một file tới hdfs trước nha mấy thằng mặt lồng. sau hồi test thì tau phát hiện ra không những không write đc đến hdfs mà còn đọc méo đc lun. éo hiểu kiểu gì?
</div>
<div class="alert alert-block alert-success">
    1. hiện tại tao chưa tìm đc giải pháp nên tao tạm thời tạo pyspark df from df normal nha.!
<div>

In [ ]:
df = ss.read.csv(path="/work/data/bank.csv", sep=",", header=True)
df

In [ ]:
# # Write file bank.csv to hdfs
# from hdfs import InsecureClient
# client = InsecureClient('http://namenode:9870')
# with client.write('/bank.csv', encoding = 'utf-8') as writer:
#     df.to_csv(writer)

In [ ]:
# Load Data to Spark Dataframe
print(df.printSchema())
df.count()

In [ ]:
# Drop Unnessary Featur/work/Day and Month)
df = df.drop('day', 'month')
# ss.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [ ]:
df.limit(10).toPandas().head() 

In [ ]:
# Helper function to select features to scale given their skew
def select_features_to_scale(df=df, lower_skew=-2, upper_skew=2, dtypes='int32', drop_cols=['']):
    
    # Empty Selected Feature List for Output
    selected_features = []
    
    # Select Features to Scale based on Inputs ('in32' type, drop 'ID' columns or others, skew bounds)
    feature_list = list(df.toPandas().select_dtypes(include=[dtypes]).columns.drop(drop_cols))
    
    # Loop through 'feature_list' to select features based on Kurtosis / Skew
    for feature in feature_list:

        if df.toPandas()[feature].kurtosis() < -2 or df.toPandas()[feature].kurtosis() > 2:
            
            selected_features.append(feature)
    
    # Return feature list to scale
    return selected_features

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = ss.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [ ]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
tokenizer

In [ ]:
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)

In [ ]:
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()